In [1]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
from tqdm import tqdm
from time import sleep
import pandas as pd
import numpy as np
import torch
import shutil
import os

In [2]:
train_image_dir = 'E:\\DL_datasets\\cifar10\\preprocess_train'
test_image_dir = 'E:\\DL_datasets\\cifar10\\test'

train_transform = transforms.Compose([
    transforms.Resize(40),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

train_dataset = ImageFolder(train_image_dir, transform=train_transform)
test_dataset = ImageFolder(test_image_dir, transform=test_transform)

In [46]:
len(train_dataset)

50000

In [48]:
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [40000, 10000])

In [51]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [52]:
def conv3x3(in_channels, out_channels, kernel_size=3, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=1, bias=False)

In [53]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride=stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels, stride=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        if in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels))
        else:
            self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        
        out += residual
        out = self.relu(out)
        return out

In [54]:
test_net = ResidualBlock(64, 128, 2)
test_x = Variable(torch.zeros(1, 64, 96, 96))
print('input:{}'.format(test_x.shape))

test_y = test_net(test_x)
print('output:{}'.format(test_y.shape))

input:torch.Size([1, 64, 96, 96])
output:torch.Size([1, 128, 48, 48])


In [55]:
class ResNet(nn.Module):
    def __init__(self, block, num_classes=10):
        super(ResNet, self).__init__()  # b, 3, 32, 32
        self.first = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True), 
            nn.MaxPool2d(3, 1, 1)) # b, 64, 
        self.layer1 = self.make_layer(block, 64, 64, 3, 1)
        self.layer2 = self.make_layer(block, 64, 128, 4, 2)
        self.layer3 = self.make_layer(block, 128, 256, 6, 2)
        self.layer4 = self.make_layer(block, 256, 512, 3, 2)
        self.avg_pool = nn.AvgPool2d(2)
        self.fc = nn.Linear(512, 10)
        
    def make_layer(self, block, in_channels, out_channels, block_num, stride):
        layers = []
        layers.append(block(in_channels, out_channels, stride))
        for i in range(block_num - 1):
            layers.append(block(out_channels, out_channels, 1))
        return nn.Sequential(*layers)
        
    def forward(self, x):
        out = self.first(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        return out

In [ ]:
batch_size = 64
learning_rate = 1e-2
epochs = 50

model = ResNet(ResidualBlock)
if torch.cuda.is_available():
    model.cuda()
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train_losses = []
train_accs = []
val_losses = []
val_accs = []
for epoch in range(epochs):
    train_loss = 0.0
    train_acc = 0
    for data, label in tqdm(train_loader):
        data = data.view(data.size(0), 3, 32, 32)
        if torch.cuda.is_available():
            data = Variable(data).cuda()
            label = Variable(label).cuda()
        else:
            data = Variable(data)
            label = Variable(label)
        
        out = model(data)
        loss = criterion(out, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.data.item() * label.size(0)
        _, pred = torch.max(out, 1)
        train_acc += (pred == label).sum().item()
    
    val_loss = 0.0
    val_acc = 0
    for data, label in val_loader:
        data = data.view(data.size(0), 3, 32, 32)
        if torch.cuda.is_available():
            data = Variable(data).cuda()
            label = Variable(label).cuda()
        else:
            data = Variable(data)
            label = Variable(label)
        
        out = model(data)
        loss = criterion(out, label)
        val_loss += loss.data.item() * label.size(0)
        _, pred = torch.max(out, 1)
        val_acc += (pred == label).sum().item()
    
    train_losses.append(train_loss/len(train_dataset))
    train_accs.append(train_acc/len(train_dataset))
    val_losses.append(val_loss/len(val_dataset))
    val_accs.append(val_acc/len(val_dataset))
    print("Epoch [{}/{}], train_loss:{:.6f}, train_acc:{:.6f}; val_loss:{:.6f}, val_acc:{:.6f}".format(epoch+1, epochs, 
                                            train_loss/len(train_dataset), train_acc/len(train_dataset),
                                            val_loss/len(val_dataset), val_acc/len(val_dataset)))

100%|███████████████████████████████| 625/625 [02:45<00:00,  3.78it/s]


Epoch [1/50], train_loss:2.670547, train_acc:0.420150; val_loss:1.960924, val_acc:0.502700


100%|███████████████████████████████| 625/625 [02:36<00:00,  4.00it/s]


Epoch [2/50], train_loss:1.660545, train_acc:0.553150; val_loss:1.444959, val_acc:0.592800


100%|███████████████████████████████| 625/625 [02:44<00:00,  3.80it/s]


...

100%|███████████████████████████████| 625/625 [02:39<00:00,  3.93it/s]


Epoch [44/50], train_loss:0.213097, train_acc:0.926900; val_loss:0.599601, val_acc:0.824600


100%|███████████████████████████████| 625/625 [02:39<00:00,  3.93it/s]


Epoch [45/50], train_loss:0.209826, train_acc:0.926600; val_loss:0.639478, val_acc:0.818600


100%|███████████████████████████████| 625/625 [02:44<00:00,  3.80it/s]


In [17]:
model.eval()
result = torch.LongTensor()
for data, label in tqdm(test_loader):
    if torch.cuda.is_available():
        data = Variable(data).cuda()
        label = Variable(label).cuda()
    else:
        data = Variable(data)
        label = Variable(label)
    
    out = model(data)
    _, pred = torch.max(out, 1)
    result = torch.cat((result, pred.cpu()), dim=0)

100%|█████████████████████████████| 4688/4688 [09:11<00:00,  8.50it/s]


In [18]:
result

tensor([4, 6, 2,  ..., 9, 6, 6])

In [34]:
images = os.listdir('E:\\DL_datasets\\cifar10\\test\\other')
result_df = pd.DataFrame({'id':images, 'label':result})

In [35]:
result_df.head()

,id,label
0,1.png,4
1,10.png,6
2,100.png,2
3,1000.png,4
4,10000.png,6


In [45]:
result_df['id'] = result_df.id.apply(lambda x: x.split('.')[0])
labels = train_dataset.classes
result_df['label'] = result_df.label.apply(lambda x: labels[x])
result_df.to_csv('20190919_submission.csv', index=False)

In [27]:
print(train_dataset.classes)
train_dataset.class_to_idx

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


{'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

参考链接：
1. PyTorch识别CIFAR10：https://www.cnblogs.com/zhengbiqing/p/10432169.html